In [55]:
#ensembling
from mlxtend.classifier import StackingCVClassifier
import numpy as np
import pandas as pd
import sys, os, glob
import imp
import seaborn as sns
sys.path.append('./src/')
import common_utils,my_ensembler, feature_utils, defines, model_utils, my_bert
from sklearn.neighbors import KNeighborsClassifier
from sklearn_crfsuite import scorers, CRF
from sklearn.pipeline import Pipeline
import json
from operator import itemgetter
from sklearn_crfsuite.utils import flatten
from sklearn_crfsuite.metrics import sequence_accuracy_score
from sklearn.linear_model import LogisticRegression
import scipy
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

### Define static parameters for data

In [14]:
dir_name='reparse'
docs_map_name = 'scaled.binary.5_stop_docs_map.json'
crf_params_name='.802_21.07_07:26_best_params.json'
ens_crf_db_name   = 'ensemble_crf_cv_db_all'
ens_lg_db_name = 'ensemble_lg_cv_db_all'
cv_crf_bert_name = 'cv_crf_bert_all'
cv_split_name='cv_splits_10'
per_par=True
seq_len=3
step=3
test_doc_num=8
n_splits=10
seed=42

### Load docs map

In [5]:
imp.reload(common_utils)
docs_map=common_utils.get_docs_map(dir_name,docs_map_name,per_par,seq_len,step)

### Use same splits as for CRF & BERT (10 x 10)

In [15]:
x=np.arange(1,81)
cv_splits_10=model_utils.get_test_train_splits(x,test_doc_num,n_splits,seed)
cv_splits_10

{0: {'test': array([ 4, 14, 15, 18, 29, 32, 36, 70]),
  'train': array([ 1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 16, 17, 19, 20, 21,
         22, 23, 24, 25, 26, 27, 28, 30, 31, 33, 34, 35, 37, 38, 39, 40, 41,
         42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58,
         59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76,
         77, 78, 79, 80])},
 1: {'test': array([ 5,  6, 13, 34, 37, 62, 73, 80]),
  'train': array([ 1,  2,  3,  4,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20,
         21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 38, 39,
         40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
         57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75,
         76, 77, 78, 79])},
 2: {'test': array([ 7, 20, 39, 47, 48, 54, 57, 58]),
  'train': array([ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3

In [81]:
n_splits=10
gsf = model_utils.GroupSplitFold(n_splits=n_splits, n_groups=test_doc_num)

for tr,ts in gsf.split(X=x, groups=x, seed=seed):
    print(ts)

[3, 13, 14, 17, 28, 31, 35, 69]
[4, 5, 12, 33, 36, 61, 72, 79]
[6, 19, 38, 46, 47, 53, 56, 57]
[0, 18, 23, 26, 43, 55, 70, 74]
[11, 20, 25, 32, 40, 49, 75, 76]
[7, 10, 15, 39, 50, 52, 62, 71]
[8, 24, 29, 37, 48, 54, 58, 73]
[2, 9, 27, 30, 41, 44, 65, 67]
[1, 22, 42, 45, 60, 64, 66, 68]
[16, 21, 34, 51, 59, 63, 77, 78]


In [34]:
X,y,groups=model_utils.get_X_y_by_doc_indices(docs_map,list(docs_map.keys()),seq_len,step)

In [42]:
tot_len=0
check=X
for l in check:
    if isinstance(l,list):
        tot_len+=len(check)
tot_len

25010001

In [64]:
rs = {}

In [72]:
global iterr

In [75]:
def increment_iter():
    global iterr
    iterr+=1
    print('шtter {}'.format(iterr))

In [82]:
%%time
# define fixed parameters and parameters to search
increment_iter()
crf = CRF(
    max_iterations=100,
    all_possible_transitions=True,
    algorithm='lbfgs'
)
c1_param_range = np.arange(0.798,0.802,0.001)
c2_param_range = np.arange(0.0078,0.0082,0.0001)

params_space = {
    'c1': c1_param_range,
    'c2': c2_param_range,
    'min_freq': np.arange(1,4,1),
#      'epsilon':scipy.stats.expon(scale=0.00005),
#      'delta':scipy.stats.expon(scale=0.00005)
}

labels = ['not_nar','is_nar']
# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search

rs[iterr] = GridSearchCV(crf, params_space,
                        cv=gsf,
                        verbose=3,
                        n_jobs=-1,
                        scoring=f1_scorer)

rs[iterr].fit(X=X, y=y,groups=groups)

tter 4
Fitting 10 folds for each of 75 candidates, totalling 750 fits
CPU times: user 12min 16s, sys: 55.9 s, total: 13min 12s
Wall time: 14min 10s


GridSearchCV(cv=<model_utils.GroupSplitFold object at 0x2b4bb6cc0e20>,
             estimator=CRF(algorithm='lbfgs', all_possible_transitions=True,
                           max_iterations=100),
             n_jobs=-1,
             param_grid={'c1': array([0.798, 0.799, 0.8  , 0.801, 0.802]),
                         'c2': array([0.0078, 0.0079, 0.008 , 0.0081, 0.0082]),
                         'min_freq': array([1, 2, 3])},
             scoring=make_scorer(flat_f1_score, average=weighted, labels=['not_nar', 'is_nar']),
             verbose=3)

In [80]:
# grid search
print('best params:', rs[iterr].best_params_)
print('best CV score:', rs[iterr].best_score_)
print('model size: {:0.2f}M'.format(rs[iterr].best_estimator_.size_ / 1000000))

best params: {'c1': 0.799, 'c2': 0.008199999999999997, 'min_freq': 3}
best CV score: 0.7956883877279403
model size: 0.28M


In [78]:
rs[0].best_params_,rs[0].best_score_

({'c1': 0.8, 'c2': 0.008, 'min_freq': 3}, 0.8145372106942936)

In [79]:
imp.reload(common_utils)
common_utils.save_best_params(rs[iterr].best_params_, rs[iterr].best_score_,dir_name)

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/.815_27.07_11:47_best_params.json.json


In [38]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.8102417978873656, 'c2': 0.003873793440565451, 'delta': 0.000216707316979366, 'epsilon': 7.397418881207557e-05, 'min_freq': 7}
best CV score: 0.7860289715476777
model size: 0.29M


In [44]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.7376072630217567, 'c2': 0.003846463275689818, 'delta': 2.2194633620811626e-05, 'epsilon': 6.157505230222483e-06, 'min_freq': 1}
best CV score: 0.7834174342882456
model size: 0.44M


In [47]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.9091855366176487, 'c2': 0.05630278956926933, 'delta': 6.64420665018946e-05, 'epsilon': 1.1727392993109942e-05, 'min_freq': 3}
best CV score: 0.7876660388159731
model size: 0.32M


In [52]:
import json
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d.%m_%H:%M")
print("date and time =", dt_string)
score="{:.3f}".format(rs.best_score_).lstrip('0')
score
json_path = os.path.join(os.getcwd(),defines.PATH_TO_DFS,dir_name,"{}_{}_best_params.json".format(score,dt_string))

best_params_pyth = common_utils.convert_to_python_types(rs.best_params_)
json_path
with open(json_path, 'w') as fp:
    json.dump(best_params_pyth, fp)

date and time = 26.07_17:35


In [53]:
json_path

'/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/.788_26.07_17:35_best_params.json'